In [2]:
import mysql.connector as mysql
import pandas as pd

In [3]:
db = mysql.connect(host = 'localhost',user = 'root',passwd = '***********',database = 'Countries')

In [4]:
cursor = db.cursor()

In [10]:
def modificar_paises(lista):
    if type(lista) != list:
        lista = lista['Entity']
    paises_mod = []
    for pais in lista:
        for letra in pais:
            if letra == ' ' or letra == '-' or letra == '(' or letra == ')':
                index = pais.index(letra)
                pais = pais[:index] + '_' + pais[index + 1:]
            if letra == "'" or letra == ':':
                index = pais.index(letra)
                pais = pais[:index] + pais[index + 1:]
            if letra == '&':
                index = pais.index(letra)
                pais = pais[:index] + 'and' + pais[index + 1:]
        if pais not in paises_mod:
            paises_mod.append(pais)
    return paises_mod

In [6]:
class Tabela:
    def __init__(self,cursor,nome,list_atributos,list_tipos):
        self.atributos = list_atributos
        self.nome = nome
        self.cursor = cursor
        try:
            comando = 'SELECT * FROM ' + self.nome + ';'
            self.cursor.execute(comando)
            self.data = self.cursor.fetchall()
        except mysql.ProgrammingError:
            atributos = '('
            for i in range(len(list_atributos)):
                atributos += list_atributos[i] + ' ' + list_tipos[i] + ','
            atributos = atributos[:-1] + ')'
            comando = 'CREATE TABLE ' + nome + ' ' + atributos + ';'
            #print(comando)
            self.cursor.execute(comando)

    def inserir_dados_double(self,nomes_atributos,lista_dados):
        nomes = '('
        for nome in nomes_atributos:
            nomes += nome + ','
        nomes = nomes[:-1] + ')'
        lista = '('
        for dado in lista_dados:
            lista += str(dado) + ','
        lista = lista[:-1] + ')'
        comando = 'INSERT INTO ' + self.nome + ' ' + nomes + ' VALUES ' + lista + ';'
        self.cursor.execute(comando)
        self.select()
        
    def clear(self):
        comando = 'DELETE FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        self.select()
        
    def select(self):
        comando = 'SELECT * FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        self.data = self.cursor.fetchall()
        return self.data
        
    def coluna(self,nome):
        comando = 'SELECT ' + nome + ' FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        col = self.cursor.fetchall()
        comando = 'SELECT ' + self.atributos[0] + ' FROM ' + self.nome + ';'
        self.cursor.execute(comando)
        ind = self.cursor.fetchall()
        mapa = dict()
        for i in range(len(ind)):
            mapa[ind[i][0]] = col[i][0]
        return mapa
    
    def get_colunas(self,lista):
        mapa = dict()
        for item in lista:
            mapa[item] = self.coluna(item)
        return mapa

In [7]:
df = pd.read_csv('gdp-per-capita-worldbank.csv')

In [11]:
paises_mod = modificar_paises(df)
tipos = ['INT']
for _ in range(len(paises_mod)):
    tipos.append('DOUBLE')

atributos = ['Ano'] + paises_mod

In [13]:
tabela1 = Tabela(cursor,'PIB_per_capita',atributos,tipos)

In [174]:
def inserir_dados(df,tabela):
    for ano in range(1990,2021):
        atrib = ['Ano']
        dados = [ano]
        for i in range(len(df)):
            if ano == df.iloc[i]['Year']:
                dados.append(df.iloc[i]['GDP per capita, PPP (constant 2017 international $)'])
                atrib.append(df.iloc[i]['Entity'])
        atrib_mod = modificar_paises(atrib)
        tabela.inserir_dados_double(atrib_mod,dados)

In [175]:
inserir_dados(df,tabela1)

In [14]:
colunas = tabela1.get_colunas(atributos[1:3])

In [191]:
db.commit()